In [1]:
KEEP_DRIVER_OPEN = False
SHOW_UI = False

In [2]:
import sys
sys.path.append('../Storage')

In [3]:
import datetime

# Get the current date and time
current_datetime = datetime.datetime.now()

# Format and print the current date and time
print("Last session Date and Time:", current_datetime)

Last session Date and Time: 2024-05-26 20:39:56.536709


In [4]:
!pip3 install -r ../requirements.txt

In [5]:
try:
    basestring
except NameError:
    basestring = str

from datetime import datetime
from decimal import Decimal
from future.utils import iteritems
import dateutil.parser

class BaseModel(object):

    """ Base class for other models. """
    
    def __init__(self, **kwargs):
        self._default_params = {}

    @classmethod
    def _NewFromJsonDict(cls, data, **kwargs):
        if kwargs:
            for key, val in kwargs.items():
                data[key] = val
        return cls(**data)

class Book(BaseModel):
    """A class that represents the Bitso orderbook and it's limits"""

    def __init__(self, **kwargs):
        self._default_params = {
            'symbol': kwargs.get('book'),
            'minimum_amount': Decimal(kwargs.get('minimum_amount')),
            'maximum_amount': Decimal(kwargs.get('maximum_amount')),
            'minimum_price': Decimal(kwargs.get('minimum_price')),
            'maximum_price': Decimal(kwargs.get('maximum_price')),
            'minimum_value': Decimal(kwargs.get('minimum_value')),
            'maximum_value': Decimal(kwargs.get('maximum_value'))
        }
        
        for (param, val) in self._default_params.items():
            setattr(self, param, val)

    def __repr__(self):
        return "Book(symbol={symbol})".format(symbol=self.symbol)
    
class AvailableBooks(BaseModel):
    """A class that represents Bitso's orderbooks"""
    def __init__(self, **kwargs):
        self.books = []
        for ob in kwargs.get('payload'):
            self.books.append(ob['book'])
            setattr(self, ob['book'], Book._NewFromJsonDict(ob))

    def __repr__(self):
        return "AvilableBooks(books={books})".format(books=','.join(self.books))


In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

#
#The MIT License (MIT)
#
#Copyright (c) 2016 Mario Romero 
#
#Permission is hereby granted, free of charge, to any person obtaining a copy
#of this software and associated documentation files (the "Software"), to deal
#in the Software without restriction, including without limitation the rights
#to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#copies of the Software, and to permit persons to whom the Software is
#furnished to do so, subject to the following conditions:
#
#The above copyright notice and this permission notice shall be included in all
#copies or substantial portions of the Software.
#
#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
#AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
#SOFTWARE.

from __future__ import absolute_import

import hashlib
import hmac
import json
import time
import requests

from future.utils import iteritems

try:
    from urllib.parse import urlparse, urlencode
except ImportError:
    from urlparse import urlparse
    from urllib import urlencode

def current_milli_time():
    nonce =  str(int(round(time.time() * 1000000)))
    return nonce

class ApiError(Exception):
    pass

class ApiClientError(Exception):
    pass

class Api(object):
    """A python interface for the Bitso API

    Example usage:
      To create an instance of the bitso.Api class, without authentication:
      
        >>> import bitso
        >>> api = bitso.Api()
      
      To get the Bitso price ticker:
      
        >>> ticker = api.ticker()
        >>> print ticker.ask
        >>> print ticker.bid

      To use the private endpoints, initiate bitso.Api with a client_id,
      api_key, and api_secret (see https://bitso.com/developers?shell#private-endpoints):
      
        >>> api = bitso.Api(API_KEY, API_SECRET)
        >>> balance = api.balance()
        >>> print balance.btc_available
        >>> print balance.mxn_available
    """
    
    def __init__(self, key=None, secret=None, timeout=0):
        """Instantiate a bitso.Api object.
        
        Args:
          key:
            Bitso API Key 
          secret:
            Bitso API Secret

  
        """
        self.base_url_v2 = "https://bitso.com/api/v2"
        self.base_url = "https://bitso.com/api/v3"
        self.key = key
        self._secret = secret
        self.timeout = timeout

    def available_books(self):
        """
        Returns:
          A list of bitso.AvilableBook instances
        """
        url = '%s/available_books/' % self.base_url
        resp = self._request_url(url, 'GET')
        return AvailableBooks._NewFromJsonDict(resp)
    
    def _build_auth_payload(self):
        parameters = {}
        parameters['key'] = self.key
        parameters['nonce'] = str(int(time.time()))
        msg_concat = parameters['nonce']+self.client_id+self.key
        parameters['signature'] = hmac.new(self._secret.encode('utf-8'),
                                           msg_concat.encode('utf-8'),
                                           hashlib.sha256).hexdigest()
        return parameters

    def _build_auth_header(self, http_method, url, json_payload=''):
        if json_payload == {} or json_payload=='{}':
            json_payload = ''
        url_components = urlparse(url)
        request_path = url_components.path
        if url_components.query != '':
            request_path+='?'+url_components.query
        nonce = current_milli_time()
        msg_concat = nonce+http_method.upper()+request_path+json_payload
        signature = hmac.new(self._secret.encode('utf-8'),
                                 msg_concat.encode('utf-8'),
                                 hashlib.sha256).hexdigest()
        return {'Authorization': 'Bitso %s:%s:%s' % (self.key, nonce, signature)}

    
    def _request_url(self, url, verb, params=None, private=False):
        headers=None
        if params == None:
            params = {}
        params = {k: v.decode("utf-8") if isinstance(v, bytes) else v for k, v in params.items()}
        if private:
            headers = self._build_auth_header(verb, url, json.dumps(params))
        if verb == 'GET':
            url = self._build_url(url, params)
            if private:
                headers = self._build_auth_header(verb, url)
            try:
                resp = requests.get(url, headers=headers, timeout=self.timeout)
            except requests.RequestException as e:
                raise
        elif verb == 'POST':
            try:
                resp = requests.post(url, json=params, headers=headers, timeout=self.timeout)
            except requests.RequestException as e:
                raise
        elif verb == 'DELETE':
            try:
                resp = requests.delete(url, headers=headers, timeout=self.timeout)
            except requests.RequestException as e:
                raise
        content = resp.content
        data = self._parse_json(content if isinstance(content, basestring) else content.decode('utf-8'))
        return data

    def _build_url(self, url, params):
        if params and len(params) > 0:
            url = url+'?'+self._encode_parameters(params)
        return url

    def _encode_parameters(self, parameters):
        if parameters is None:
            return None
        else:
            param_tuples = []
            for k,v in parameters.items():
                if v is None:
                    continue
                if isinstance(v, (list, tuple)):
                    for single_v in v:
                        param_tuples.append((k, single_v))
                else:
                    param_tuples.append((k,v))
            return urlencode(param_tuples)


         
    def _parse_json(self, json_data):
        try:
            data = json.loads(json_data)
            self._check_for_api_error(data)
        except:
            raise
        return data

    def _check_for_api_error(self, data):
        if data['success'] != True:
            raise ApiError(data['error'])
        if 'error' in data:
            raise ApiError(data['error'])
        if isinstance(data, (list, tuple)) and len(data)>0:
            if 'error' in data[0]:
                raise ApiError(data[0]['error'])

In [7]:
api = Api(timeout=5)
avb_books = api.available_books()
print(f"Total Available Books: {len(avb_books.books)}")
print(f"Available Books: {avb_books.books}")

Total Available Books: 97
Available Books: ['btc_mxn', 'eth_mxn', 'xrp_mxn', 'ltc_mxn', 'bch_mxn', 'tusd_btc', 'tusd_mxn', 'mana_mxn', 'bat_mxn', 'btc_dai', 'dai_mxn', 'btc_usd', 'xrp_usd', 'eth_usd', 'btc_brl', 'btc_usdt', 'usd_mxn', 'usd_brl', 'mana_usd', 'ltc_usd', 'uni_usd', 'aave_usd', 'chz_usd', 'dydx_usd', 'yfi_usd', 'snx_usd', 'matic_usd', 'mkr_usd', 'enj_usd', 'ftm_usd', 'crv_usd', 'gala_usd', 'ada_usd', 'grt_usd', 'ape_usd', 'sushi_usd', 'omg_usd', 'dot_usd', 'qnt_usd', 'doge_usd', 'usd_cop', 'trx_usd', 'ldo_usd', 'xlm_usd', 'xrp_brl', 'usdt_brl', 'paxg_usd', 'shib_brl', 'avax_usd', 'ltc_brl', 'bat_usd', 'bch_usd', 'usdt_mxn', 'eth_btc', 'atom_usd', 'near_usd', 'usd_usdt', 'avax_mxn', 'sol_mxn', 'trx_mxn', 'eth_usdt', 'arb_usd', 'xrp_usdt', 'sand_usd', 'link_usd', 'bal_usd', 'axs_usd', 'ada_brl', 'usd_ars', 'btc_ars', 'dai_ars', 'eth_ars', 'usdt_ars', 'shib_usd', 'eur_mxn', 'eur_brl', 'pepe_usd', 'btc_cop', 'eth_cop', 'usdt_cop', 'xrp_cop', 'sol_usd', 'link_brl', 'matic_mxn',

In [8]:
usd_books = [book for book in avb_books.books if 'mxn' not in book]
usd_books = [book for book in usd_books if 'brl' not in book]
usd_books = [book for book in usd_books if 'cop' not in book]
usd_books = [book for book in usd_books if 'ars' not in book]
print(f"Total USD Available Books: {len(usd_books)}")
print(f"USD Available Books: {usd_books}")

Total USD Available Books: 56
USD Available Books: ['tusd_btc', 'btc_dai', 'btc_usd', 'xrp_usd', 'eth_usd', 'btc_usdt', 'mana_usd', 'ltc_usd', 'uni_usd', 'aave_usd', 'chz_usd', 'dydx_usd', 'yfi_usd', 'snx_usd', 'matic_usd', 'mkr_usd', 'enj_usd', 'ftm_usd', 'crv_usd', 'gala_usd', 'ada_usd', 'grt_usd', 'ape_usd', 'sushi_usd', 'omg_usd', 'dot_usd', 'qnt_usd', 'doge_usd', 'trx_usd', 'ldo_usd', 'xlm_usd', 'paxg_usd', 'avax_usd', 'bat_usd', 'bch_usd', 'eth_btc', 'atom_usd', 'near_usd', 'usd_usdt', 'eth_usdt', 'arb_usd', 'xrp_usdt', 'sand_usd', 'link_usd', 'bal_usd', 'axs_usd', 'shib_usd', 'pepe_usd', 'sol_usd', 'sol_usdt', 'algo_usd', 'comp_usd', 'tigres_usd', 'bar_usd', 'psg_usd', 'lrc_usd']


In [9]:
usd_books = [book.replace('_', '-') for book in usd_books]
print(f"USD Available Books: {usd_books}")

USD Available Books: ['tusd-btc', 'btc-dai', 'btc-usd', 'xrp-usd', 'eth-usd', 'btc-usdt', 'mana-usd', 'ltc-usd', 'uni-usd', 'aave-usd', 'chz-usd', 'dydx-usd', 'yfi-usd', 'snx-usd', 'matic-usd', 'mkr-usd', 'enj-usd', 'ftm-usd', 'crv-usd', 'gala-usd', 'ada-usd', 'grt-usd', 'ape-usd', 'sushi-usd', 'omg-usd', 'dot-usd', 'qnt-usd', 'doge-usd', 'trx-usd', 'ldo-usd', 'xlm-usd', 'paxg-usd', 'avax-usd', 'bat-usd', 'bch-usd', 'eth-btc', 'atom-usd', 'near-usd', 'usd-usdt', 'eth-usdt', 'arb-usd', 'xrp-usdt', 'sand-usd', 'link-usd', 'bal-usd', 'axs-usd', 'shib-usd', 'pepe-usd', 'sol-usd', 'sol-usdt', 'algo-usd', 'comp-usd', 'tigres-usd', 'bar-usd', 'psg-usd', 'lrc-usd']


In [10]:
def from_book(book):
    cum = []
    start = False
    for usd_book in usd_books:
        if usd_book == book:
            start = True
        if start:
            cum.append(usd_book)
    print(f"From chosen USD Available Book: {cum}")
    return cum

In [11]:
import pgConn
import PostgresSQL_table_queries

pg_conn = pgConn.PgConn(tablename="historical", dbname="cryptostocks", user="postgres")
pg_conn.init_db(PostgresSQL_table_queries.HISTORICAL_CRYPTO_STOCKS_TABLE_QUERY)
# pg_conn.set_table("another_custom_table_name")
# pg_conn.save_to_postgres(row_data, header)
# # Perform other operations using pg_conn
# pg_connpg_conn = super.initDB('postgres', "historical", "cryptostocks", "postgres", PostgresSQL_table_queries.HISTORICAL_CRYPTO_STOCKS_TABLE_QUERY).close_connection()

Connection to the database successful!
Table name set to: historical
Table 'historical' already exists.


<connection object at 0x1154eece0; dsn: 'user=postgres password=xxx dbname=cryptostocks host=localhost port=5432', closed: 0>

In [12]:
import CloudStorage as cs
import boto3
import os

def store_to_s3(bucket_name, folder_name):
    # Bucket name and folder paths
    local_file_path = "path/to/local/file.txt"
    #s3_file_path = f"{folder_name}/file.txt"
    current_directory = os.getcwd()
    s3_file_path = f"{current_directory}/{folder_name}/file.txt"
    
    # Create the bucket and folder if they don't exist
    cs.create_bucket(bucket_name)
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    bucket.put_object(Key=s3_file_path, Body="")  # Create an empty object to create the folder

    # Upload the file to S3
    cs.upload_file_to_s3(bucket_name, local_file_path, s3_file_path)

In [13]:
import time
import json
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By

In [14]:
contents = []
url = f'https://finance.yahoo.com/lookup'
xpath = "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/div/ul/li[1]/div/div/div[2]/h3/a"

In [15]:
MAIN_SECTION_STOCK_DATA_HTML_EL = "/html/body/div[1]/main/section/section/section"
HISTORIAL_DATA_BTN = "/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[4]/a"

STOCKS_HTML_TABLE = "/html/body/div[1]/main/section/section/section/article/div[1]/div[3]/table"
STOCKS_HTML_TABLE_BODY = "/html/body/div[1]/main/section/section/section/article/div[1]/div[3]/table/tbody"

NO_RESULTS_FOUND_HTML_SPAN_EL = "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/section/div/div/span/span"

In [16]:
from datetime import datetime

def parse_date(date_str):
    # Convert the month name to a numerical representation using a dictionary
    month_dict = {
        "Jan": "01",
        "Feb": "02",
        "Mar": "03",
        "Apr": "04",
        "May": "05",
        "Jun": "06",
        "Jul": "07",
        "Aug": "08",
        "Sep": "09",
        "Oct": "10",
        "Nov": "11",
        "Dec": "12",
    }

    date_str = date_str.replace(",", "")

    # Split the date string into month, day, and year
    month, day, year = date_str.split()

    # Get the numerical representation of the month from the dictionary
    month_number = month_dict[month]

    # Create a new date string in the format 'year-month-day' (e.g., '2023-08-01')
    formatted_date_str = f"{year}-{month_number}-{day}"

    # Parse the formatted date string to a datetime object
    parsed_date = datetime.strptime(formatted_date_str, "%Y-%m-%d")

    return parsed_date

def parse_row_data(row_data):
    try:
        date_format = '%Y-%m-%d'  # Format for parsing date strings

        # Remove commas from numeric values
        row_data = [item.replace(",", "") if isinstance(item, str) else item for item in row_data]

        # Parse elements at specific positions into desired data types
        row_data[0] = parse_date(row_data[0])
        row_data[1] = float(row_data[1])
        row_data[2] = float(row_data[2])
        row_data[3] = float(row_data[3])
        row_data[4] = float(row_data[4])
        row_data[5] = float(row_data[5])
        row_data[6] = int(row_data[6])
        return row_data
    except Exception as e:
        print("error during parsing data:", e, "row_data: ", row_data)

In [17]:
import csv
import os

def save_unavailable_book(book_name):
    try:
        current_directory = os.getcwd()
        unavailable_books_file = os.path.join(current_directory, "unavailable_books.csv")
        
        file_exists = os.path.isfile(unavailable_books_file)
        with open(unavailable_books_file, "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            if not file_exists:
                writer.writerow(["book"])  # Add header if the file is newly created
            writer.writerow([book_name])
        print(f"Book '{book_name}' added to unavailable_books.csv")
    except Exception as e:
        print(f"Error while saving book '{book_name}' to CSV: {e}")

In [18]:
def get_dynamic_url(ticker, period1=1410825600, period2=1690675200, interval="1d",adjclose="true"):
    return f'https://finance.yahoo.com/quote/{ticker.upper()}/history?period1={period1}&period2={period2}&interval={interval}&filter=history&frequency={interval}&includeAdjustedClose={adjclose}'

def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
def is_at_bottom(driver):
    lastHeight = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("var scrollingElement = (document.scrollingElement || document.body);scrollingElement.scrollTop = scrollingElement.scrollHeight;")
        height = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script("window.scrollTo(0, " + str(height) + ");")
        time.sleep(2)
        if lastHeight == height:
            print("scrolling down task finished")
            break
        lastHeight = height

def printInnerHTML(xpath):
    # Get the inner HTML of the specific element
    specific_element = driver.find_element(By.XPATH, xpath)
    
    inner_html = specific_element.get_attribute('innerHTML')

    # Print the inner HTML of the specific element
    print(inner_html)


def printXPathAndClass(el):
    # Get the XPath of the element
    element_xpath = el.get_attribute("xpath")
    print("Element XPath:", element_xpath)
    
    # Get the class attribute of the element
    element_class = el.get_attribute("class")
    print("Element Class:", element_class)
        
def check_tab_header(driver):
    try:
        element = driver.find_element(By.XPATH, '//*[@id="quote-nav"]')
        #tab = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[3]/a')
        return True
    except Exception as e:
        print(f"financial header or historical data tab does not exist: {e}")
        return False

def check_html_el_exist(driver, xpath):
    wait = WebDriverWait(driver, 3.0)
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        return True
    except NoSuchElementException:
        print("Element does not exist")
        return False

def nomatchresult(driver, book):
    try:
        if (driver.current_url == f"https://finance.yahoo.com/lookup?s={book.upper()}" or check_html_el_exist(driver, NO_RESULTS_FOUND_HTML_SPAN_EL)):
            print(f"no data was found for {book.upper()}")
            return True
    except Exception as nse:
            print(f"{book.upper()} book was found!")
            return False
            
def lookup_ticker(driver, ticker):
    RejectAll= driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div/div/div[1]/div/div/div/form/input')
    action = ActionChains(driver)
    action.click(on_element = RejectAll)
    action.perform()
    time.sleep(5)
    SearchBar = driver.find_element(By.ID, "yfin-usr-qry")
    SearchBar.send_keys(ticker.upper())
    SearchBar.send_keys(Keys.ENTER)

def select_historical_li(driver):
    li_historical_a = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[4]/a')
    action = ActionChains(driver)
    action.click(on_element = li_historical_a)
    action.perform()
    time.sleep(3)

def disable_ad(driver): 
    wait = WebDriverWait(driver, 3.0)
    try:
        ad_element = '//*[@id="Col1-0-Ad-Proxy"]'
        wait.until(EC.presence_of_element_located((By.XPATH, ad_element)))
        driver.execute_script("arguments[0].style.display = 'none';", ad_element)
    except Exception as e:
        print("ad element was not found")

def historical_stock_search_selector(driver):
    print("selecting historical dropdown menu")
    wait = WebDriverWait(driver, 3.0)
    try:
        selector1 = "/html/body/div[1]/main/section/section/section/article/div[1]/div[1]/div[1]" # Menu container
        wait.until(EC.presence_of_element_located((By.XPATH, selector1)))
        return selector1
    except Exception as e:
        print("selector1 for time period not found trying the second")
        printInnerHTML("/html/body/div[1]/main/section/section/section/article/")
        try:
            selector2 = "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section"
            wait.until(EC.presence_of_element_located((By.XPATH, selector2)))
            return selector2
        except Exception as e:
            print("selector2 for time period not found")
            
def select_historical(driver, time_period, freq):
    print("Assessing historical stock prices table data ...", end='', flush=True)
    # disable_ad(driver)
    wait = WebDriverWait(driver, 3.0)
    hs_se = historical_stock_search_selector(driver)
    action = ActionChains(driver)
    hs_se_button = driver.find_element(By.XPATH, f"{hs_se}/button")
    hs_se_button.click()

    try:
        '''
        TODO Add Frequency HTML button element
        '''
        hs_period_dropdown_div = ''
        if (time_period == '1d'):
            wait.until(EC.presence_of_element_located((By.XPATH, f"{hs_se}/div/div/div[2]/section/div[1]/button[1]")))
            hs_period_dropdown_div = driver.find_element(By.XPATH, f"{hs_se}/div/div/div[2]/section/div[1]/button[1]")
            hs_period_dropdown_div.click()
        elif (time_period == '5d'):
            wait.until(EC.presence_of_element_located((By.XPATH, f"{hs_se}/div/div/div[2]/section/div[1]/button[2]")))
            hs_period_dropdown_div = driver.find_element(By.XPATH, f"{hs_se}/div/div/div[2]/section/div[1]/button[2]")
            hs_period_dropdown_div.click()
        elif (time_period == '1y'):
            wait.until(EC.presence_of_element_located((By.XPATH, f"{hs_se}/div/div/div[2]/section/div[1]/button[6]")))
            hs_period_dropdown_div = driver.find_element(By.XPATH, f"{hs_se}/div/div/div[2]/section/div[1]/button[6]")
            hs_period_dropdown_div.click()
    except Exception as e:
        print("Error on select_historical(): ", e)
        print("====================================================================")
        print("Printing inner HTML")
        print("====================================================================")
        printInnerHTML(hs_se)
    
    
    wait.until(EC.presence_of_element_located((By.XPATH, STOCKS_HTML_TABLE)))
    print("Task finished")

In [19]:
DRIVER_PATH = "/chromedriver/chromedriver"
options = webdriver.ChromeOptions()
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.90 Safari/537.36")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-extensions")
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument("--start-maximized")
if not SHOW_UI:
    options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--ignore-certificate-errors')

In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [21]:
REFERENCE = 'https://finance.yahoo.com'
Header = ["reference", "book", "date", "open", "high", "low", "close", "adj_close", "volume"]
n = len(Header)
Debug = False
time_period = '1d'
frequency = 'daily'
show_row_data = True
frombook = ''
if (len(frombook) > 0):
    usd_books = from_book(frombook)

try:
    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    try:
        for book in usd_books:
            print(f'Book: {book}')
            target_url = f"https://finance.yahoo.com/quote/{book.upper()}/history?p={book.upper()}"
            print(target_url)
            driver.get(target_url)
            
            if(nomatchresult(driver, book)):
                print("skipping to next ticket")
                # save_unavailable_book(book)
                print("====================================================================")
                continue
                
            select_historical(driver, time_period, frequency)
            time.sleep(1)

            is_at_bottom(driver)
            table = driver.find_element(By.XPATH, STOCKS_HTML_TABLE_BODY)
            # Get all rows of the table
            rows = table.find_elements(By.TAG_NAME, "tr")

            # Create an empty list to store the table data
            #table_data = []
            #df_book = pd.DataFrame(table_data, columns=Header)
            # Iterate through each row
            print("Scraping raw stock prices data task started")
            for idx, row in enumerate(rows):
                if time_period == '1d' and idx > 1:
                    break
                # Get all columns (cells) of the row
                columns = row.find_elements(By.TAG_NAME, "td")
                row_data = []
                row_data = [column.text for column in columns if column.text != '-']
                if(len(row_data) != 7):
                    print("skipping to next row")
                    continue
                row_data = parse_row_data(row_data)
                row_data.insert(0, book)
                row_data.insert(0, REFERENCE)
                try:
                    if (show_row_data):
                        print("test: ", row_data)
                    pg_conn.save_to_postgres(row_data, Header)
                except Exception as e:
                    print(f"error while saving to postgres: {e}")
                #df_book = pd.concat([df_book, pd.DataFrame([row_data], columns=Header)], ignore_index=True)
            #df = pd.concat([df, df_book], ignore_index=True)
            #num_rows, num_columns = df.shape
            #last_five_rows = df.tail(3)
            print("Scraping raw stock prices data task finished")
            print("====================================================================")
        print("**All book data was scraped**")
    except NoSuchElementException as nse:
        print(nse)
        print("-----")
        print(str(nse))
        print("-----")
        print(nse.args)
        print("=====")
except TimeoutException as toe:
    print(toe)
    print("-----")
    print(str(toe))
    print("-----")
    print(toe.args)
finally:
    if(Debug):
        delete_table("historical", conn)
    pg_conn.close_connection()
if (not KEEP_DRIVER_OPEN):
    driver.close()

Book: tusd-btc
https://finance.yahoo.com/quote/TUSD-BTC/history?p=TUSD-BTC
TUSD-BTC book was found!
Assessing historical stock prices table data ...selecting historical dropdown menu
Error on select_historical():  Message: 
Stacktrace:
0   chromedriver                        0x000000010ea4a6c8 chromedriver + 6149832
1   chromedriver                        0x000000010ea41cea chromedriver + 6114538
2   chromedriver                        0x000000010e4ced5c chromedriver + 400732
3   chromedriver                        0x000000010e51baa5 chromedriver + 715429
4   chromedriver                        0x000000010e51bbf1 chromedriver + 715761
5   chromedriver                        0x000000010e560dd4 chromedriver + 998868
6   chromedriver                        0x000000010e53ecdd chromedriver + 859357
7   chromedriver                        0x000000010e55e0db chromedriver + 987355
8   chromedriver                        0x000000010e53ea53 chromedriver + 858707
9   chromedriver                 

JavascriptException: Message: javascript error: {"status":32,"value":"Unable to locate an element with the xpath expression /html/body/div[1]/main/section/section/section/article/ because of the following error:\nSyntaxError: Failed to execute 'evaluate' on 'Document': The string '/html/body/div[1]/main/section/section/section/article/' is not a valid XPath expression."}
  (Session info: chrome-headless-shell=125.0.6422.78)
Stacktrace:
0   chromedriver                        0x000000010ea4a6c8 chromedriver + 6149832
1   chromedriver                        0x000000010ea41cea chromedriver + 6114538
2   chromedriver                        0x000000010e4ced5c chromedriver + 400732
3   chromedriver                        0x000000010e4d45a3 chromedriver + 423331
4   chromedriver                        0x000000010e4d634f chromedriver + 430927
5   chromedriver                        0x000000010e4d63fc chromedriver + 431100
6   chromedriver                        0x000000010e51b8ad chromedriver 